<a href="https://colab.research.google.com/github/wcjona/BMEN_415_Final_Project/blob/main/Image_Classification/Deep_Residual_Learnign_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install deeplake;

In [ ]:
import deeplake
from PIL import Image
import numpy as np
import os, time
import torch
from torchvision import transforms, models

In [ ]:
ds_train = deeplake.load('hub://activeloop/chest-xray-train')
ds_test = deeplake.load('hub://activeloop/chest-xray-test')
# https://datasets.activeloop.ai/docs/ml/datasets/chest-x-ray-image-dataset/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/chest-xray-train

hub://activeloop/chest-xray-train loaded successfully.

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/chest-xray-test

hub://activeloop/chest-xray-test loaded successfully.



In [ ]:
tform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((1500,1500), antialias=True),
])

In [ ]:
train_loader = ds_train.pytorch(num_workers=0, transform = {'images': tform, 'labels': None}, batch_size=4, shuffle=False, decode_method={"images": "numpy"})
test_loader = ds_test.pytorch(num_workers=0,  transform = {'images': tform, 'labels': None}, batch_size=4, shuffle=False, decode_method={"images": "numpy"})

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


In [ ]:
# Options for models: 
# https://pytorch.org/vision/stable/models.html
# RESNET18: Deep Residual Learning
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Convert model to grayscale
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Update the fully connected layer based on the number of classes in the dataset
model.fc = torch.nn.Linear(model.fc.in_features, len(ds_train.labels.info.class_names))

model.to(device)

# Specity the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.1)

In [ ]:
def train_model(model, optimizer, data_loader, device):

    model.train()

    running_loss = 0.0
    total = 0
    correct = 0
    
    for i, data in enumerate(data_loader):
        start_time = time.time()
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['images']
        labels = torch.squeeze(data['labels'])

        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
    
        if i % 10 == 0:    # print every 10 batches
            batch_time = time.time()
            speed = (batch_time-start_time)
            print('[%5d] loss: %.3f, speed: %.2f, accuracy: %.2f %%' %
                  (i, running_loss, speed, accuracy))

            running_loss = 0.0
            total = 0
            correct = 0
        if i == 100: 
          break;
    print('Finished Training')

In [ ]:
train_model(model, optimizer, train_loader, device)

[    0] loss: 0.000, speed: 59.02, accuracy: 0.00 %
[   10] loss: 0.000, speed: 53.66, accuracy: 90.00 %
[   20] loss: 0.000, speed: 53.07, accuracy: 100.00 %
[   30] loss: 0.000, speed: 53.52, accuracy: 100.00 %
[   40] loss: 0.000, speed: 52.03, accuracy: 100.00 %
[   50] loss: 0.000, speed: 52.42, accuracy: 100.00 %
[   60] loss: 0.000, speed: 52.34, accuracy: 100.00 %
[   70] loss: 0.000, speed: 52.48, accuracy: 100.00 %
[   80] loss: 0.000, speed: 52.19, accuracy: 100.00 %
[   90] loss: 0.000, speed: 50.44, accuracy: 100.00 %
[  100] loss: 0.000, speed: 50.58, accuracy: 100.00 %
Finished Training


In [ ]:
y_pred = []
y_true = []

def test_model(model, data_loader):
    model.eval()

    start_time = time.time()
    total = 0
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(data_loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data['images']
            labels = torch.squeeze(data['labels'])

            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs.float())

            _, predicted = torch.max(outputs.data, 1)
            y_pred.extend(predicted.data.cpu().numpy())
            y_true.extend(labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            accuracy = 100 * correct / total
            if i % 10 == 0: 
              print("Current Accuracy: {accuracy}")

        print('Finished Testing')
        print('Testing accuracy: %.1f %%' %(accuracy))

In [ ]:
test_model(model, test_loader)

Current Accuracy: 100.0
Current Accuracy: 100.0
Current Accuracy: 100.0
Current Accuracy: 100.0
Current Accuracy: 100.0
Current Accuracy: 100.0
Current Accuracy: 95.90163934426229
Current Accuracy: 82.3943661971831
Current Accuracy: 72.22222222222223
Current Accuracy: 64.28571428571429
Current Accuracy: 57.92079207920792
Finished Testing
Testing accuracy: 57.9 %
